**<font  size=4>因數據量大容易爆內存，寫點面向對象是個好選擇</font>**

In [1]:
import pandas as pd
import pyodbc
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")


In [ ]:
#假設我有個列表(tuple也可以)裡面有股票代號，示範只導入前十項
s_code_df = pd.read_excel("Z:/07個人/ChiehLin/TCRINEW.xlsx")
stock_ls = list(s_code_df.loc[:10,'NUM'])

In [3]:
stock_ls

[1101, 1102, 1103, 1104, 1108, 1109, 1110, 1201, 1203, 1210, 1215]

In [4]:
class get_df:
    stock_ls = str([str(x) for x in stock_ls])[1:-1]
    DB_infos = {"X01" : ("192.168.1.16", "X01", "cm", "cm1234")}
    querys = {"日融資券排行" : """SELECT  [日期]
                                        ,[股票代號]
                                        ,[股票名稱]
                                        ,[資使用率]
                                        ,[融資維持率(%%)]
                                        FROM [X01].[dbo].[日融資券排行] WITH(NOLOCK) where 股票代號 IN (%s) And 日期 >= 20140506 order by 日期 """%stock_ls ,
              "日收盤表排行" : """SELECT  [日期]
                                        ,[股票代號]
                                        ,[股票名稱]
                                        ,[開盤價]
                                        ,[最高價]
                                        ,[最低價]
                                        ,[收盤價]
                                        ,[成交金額(千)]
                                        FROM [X01].[dbo].[日收盤表排行] WITH(NOLOCK) where 股票代號 IN (%s) And 日期 >= 20140506 order by 日期 """%stock_ls , 
              "日報酬率比較表": """SELECT [日期]
                                        ,[股票代號]
                                        ,[股票名稱]
                                        ,[還原收盤價]
                                        FROM [X01].[dbo].[日報酬率比較表] WITH(NOLOCK) where 股票代號 IN (%s) And 日期 >= 20140506 order by 日期 """%stock_ls                      
                                        }
    db = "Data"   
    def db_connect(self, DB_info):
        DB_Host, DB_Name, DB_User, DB_Password = self.DB_infos[DB_info]
        print("Connecting to database using pyodbc...")
        self.db = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; SERVER=' + DB_Host + '; DATABASE=' + DB_Name + '; UID=' + DB_User + '; PWD=' + DB_Password)
        print("Succesfully Connected to database using pyodbc!")
      
    def load_data(self,*query):
        df_from_sql = pd.DataFrame()
        for i in range(len(query)):
            df =  pd.read_sql(self.querys[query[i]], self.db)
            if len(df_from_sql) == 0:
                df_from_sql = df
            else:
                df_from_sql = df_from_sql.merge(df, on = ['日期', '股票代號', '股票名稱'])    
        return df_from_sql
    
    def get_dict(self,df):
        data_dict_1 = {feature :df.pivot('日期','股票代號',feature) for feature in df.columns[3:]}
        data_dict_2 = {str(i):df[df['股票代號'] == str(i)] for i in stock_ls}
        data_dict_1.update(data_dict_2)
        return data_dict_1

In [5]:
source = get_df()

In [6]:
source.db_connect("X01")

Connecting to database using pyodbc...
Succesfully Connected to database using pyodbc!


In [7]:
#建立主要dataframe
data_frame = source.load_data( "日收盤表排行", "日融資券排行", "日報酬率比較表")

In [8]:
data_frame

,日期,股票代號,股票名稱,開盤價,最高價,最低價,收盤價,成交金額(千),資使用率,融資維持率(%),還原收盤價
0,20140506,1104,環泥,29.05,29.15,29.00,29.00,3428,3.68,159.481201,17.50
1,20140506,1109,信大,14.80,14.80,14.65,14.75,6959,2.59,190.517700,10.44
2,20140506,1203,味王,22.85,22.85,22.60,22.70,1739,9.65,153.771301,16.33
3,20140506,1103,嘉泥,16.00,16.05,15.85,16.00,2824,4.24,167.481293,10.95
4,20140506,1108,幸福,8.73,8.78,8.63,8.73,1625,7.50,157.666794,6.28
...,...,...,...,...,...,...,...,...,...,...,...
22257,20220809,1108,幸福,10.70,10.80,10.70,10.70,918,2.09,138.662292,10.70
22258,20220809,1103,嘉泥,17.65,17.90,17.65,17.80,2792,0.88,144.387100,17.80
22259,20220809,1203,味王,33.30,33.65,33.10,33.45,431,0.18,165.892700,33.45
22260,20220809,1104,環泥,21.20,21.30,21.10,21.15,14717,0.56,159.949707,21.15


In [9]:
data_dict = source.get_dict(data_frame)

In [10]:
data_dict['收盤價']

股票代號,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,1215
日期,,,,,,,,,,,
20140506,46.80,39.50,16.00,29.00,8.73,14.75,16.35,44.75,22.70,30.20,26.25
20140507,46.00,39.10,15.95,28.80,8.72,14.65,16.10,44.95,22.95,30.00,25.40
20140508,45.70,39.20,15.90,29.10,8.74,14.50,16.20,44.90,22.90,30.30,24.95
20140509,45.30,39.35,15.80,28.75,8.79,14.50,16.05,44.70,22.80,30.30,24.95
20140512,44.20,39.20,15.80,28.35,8.75,14.55,16.10,43.70,22.75,30.05,26.05
...,...,...,...,...,...,...,...,...,...,...,...
20220803,39.10,41.45,17.90,21.70,10.55,19.85,18.00,19.95,33.25,47.15,78.30
20220804,38.75,40.70,17.30,21.65,10.65,18.15,17.90,19.75,33.20,47.20,77.50
20220805,38.85,40.95,17.55,21.60,10.75,18.15,17.95,19.75,33.45,47.35,78.40


In [11]:
data_dict['1101']

,日期,股票代號,股票名稱,開盤價,最高價,最低價,收盤價,成交金額(千),資使用率,融資維持率(%),還原收盤價
8,20140506,1101,台泥,46.85,47.05,46.65,46.80,279876,0.36,164.136795,21.81
13,20140507,1101,台泥,46.60,46.65,46.00,46.00,687831,0.41,162.056793,21.43
30,20140508,1101,台泥,46.10,46.20,45.50,45.70,573230,0.49,161.901001,21.29
35,20140509,1101,台泥,45.50,45.70,44.75,45.30,599689,0.51,161.052200,21.11
52,20140512,1101,台泥,44.90,45.00,44.00,44.20,606761,0.54,158.133408,20.60
...,...,...,...,...,...,...,...,...,...,...,...
22209,20220803,1101,台泥,39.05,39.10,38.75,39.10,623078,0.55,160.391403,39.10
22226,20220804,1101,台泥,39.05,39.20,38.55,38.75,733810,0.53,159.009598,38.75
22231,20220805,1101,台泥,39.10,39.15,38.80,38.85,450508,0.53,159.522598,38.85
22248,20220808,1101,台泥,38.85,38.85,38.30,38.70,487687,0.52,158.981400,38.70
